In [ ]:
trn_trans=pd.read_csv('/kaggle/input/ieee-fraud-detection/train_identity.csv')
trn=pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv')
tst_trans=pd.read_csv('/kaggle/input/ieee-fraud-detection/test_identity.csv')
tst=pd.read_csv('/kaggle/input/ieee-fraud-detection/test_transaction.csv')
fnl_trn=trn.merge(trn_trans,on='TransactionID',how='left')
fnl_tst=tst.merge(tst_trans,on='TransactionID',how='left')
fnl_trn=fnl_trn.drop(list(fnl_trn.filter(regex='V')),1)
fnl_tst=fnl_tst.drop(list(fnl_tst.filter(regex='V')),1)
import gc
dk={}
for i in range(1,39):
    if i<10:
        dk['id-0'+str(i)]='id_0'+str(i)
    else:
        dk['id-'+str(i)]='id_'+str(i)
fnl_tst=fnl_tst.rename(columns=dk)
del([trn,tst,trn_trans,tst_trans])
gc.collect()

In [ ]:
fnl_trn['is_train']=1
fnl_tst['is_train']=0
fnl=pd.concat([fnl_trn,fnl_tst],0)
fnl=fnl.reset_index(drop=True)
gc.collect()
numerical = ["TransactionAmt", "dist1", "dist2"] + ["C" + str(i) for i in range(1, 15)] + \
            ["D" + str(i) for i in range(1, 16)] 
categorical = ["ProductCD", "card1", "card2", "card3", "card4", "card5", "card6", "addr1", "addr2",
               "P_emaildomain", "R_emaildomain",
              "DeviceInfo", "DeviceType"] + ["id_0" + str(i) for i in range(1, 10)] +\
                ["id_" + str(i) for i in range(10, 39)] + \
                 ["M" + str(i) for i in range(1, 10)]
for col in numerical:
    fnl[col]=fnl[col].fillna(fnl[col].mean())
for col in categorical:
    fnl[col]=fnl[col].fillna('nan')

fnl['days']=fnl['TransactionDT']//86400
fnl['id']=fnl['days']-fnl['D1']
fnl['id']=fnl['id'].astype(str)+fnl['P_emaildomain'].astype(str)+fnl['card1'].astype(str)+fnl['C9'].astype(str)+fnl['C11'].astype(str)+fnl['dist1'].astype(str)
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
fnl['id']=le.fit_transform(fnl['id'])
from tqdm import tqdm
for col in tqdm(numerical):
    try:
        fnl[col+'_mean']=fnl.groupby(['id'])[col].transform('mean')
        fnl[col+'_std']=fnl.groupby(['id'])[col].transform('std')
    except:
        continue
fnl=fnl.drop(['id'],1)
from tqdm import tqdm
for col in tqdm(categorical):
        print('before '+str(fnl[col].nunique()))
        fnl.loc[~fnl[col].isin(list(fnl[col].value_counts().index[:5])),col]='other'
        print('before '+str(fnl[col].nunique()))
        if col!='card1' or col !='id_02':
            fnl_dum=pd.get_dummies(fnl[col]).astype('uint8')
            fnl_dum.columns=list(np.char.add(col,np.asarray(list(fnl_dum)).astype(str)))
            fnl_dum.columns=fnl_dum.columns+'_dum'
            
            #fnl=fnl.drop([col],1)
            fnl=pd.concat([fnl,fnl_dum],1)
fnl['day']=fnl['TransactionDT']//86400
fnl['month']=fnl['day']//30
fnl['week']=fnl['day']//7

In [ ]:
import lightgbm as lgb
import gc
fnl.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in fnl.columns]
fnl=fnl.drop(['TransactionID','TransactionDT'],1)
fnl=fnl.reset_index(drop=True)
gc.collect()
fnl=fnl.fillna(-1)
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        df[col]=df[col].fillna(-1)
        df[col]=df[col].replace([np.inf,-np.inf],-1)
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df
trn=pd.read_csv('/kaggle/input/ieee-fraud-detection/train_transaction.csv',usecols=['isFraud'])
fnl['isFraud']=trn['isFraud']
fnl=fnl.loc[:,~(fnl.columns.duplicated())]

In [ ]:
from tqdm import tqdm
import gc
pds=['day','month','week']
#cols=list(fnl)

cols = ["TransactionAmt", "dist1", "dist2"] + ["C" + str(i) for i in range(1, 15)] + \
            ["D" + str(i) for i in range(1, 16)] 
for pd in pds:
  for col in tqdm(cols):
    col=str(col)
    use_col=str(col)+'_'+pd+'_ss'
#    if use_col in cols:
    fnl['a']=fnl.groupby([pd])[col].transform('max')
    fnl['b']=fnl.groupby([pd])[col].transform('min')
    fnl[use_col]=fnl[col]-fnl['b']/(fnl['a']-fnl['b'])
    use_col=str(col)+'_'+pd+'_norm'
#    if use_col in cols:
    fnl['a']=fnl.groupby([pd])[col].transform('std')
    fnl['b']=fnl.groupby([pd])[col].transform('mean')
    fnl[use_col]=(fnl[col]-fnl['a'])/fnl['b']
  
  try:
      fnl=reduce_mem_usage(fnl)
  except:
    continue
del([fnl['a'],fnl['b']])
gc.collect()

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        if col_type in ['int','float']:
            df[col]=df[col].fillna(-1)
            df[col]=df[col].replace([np.inf,-np.inf],-1)
        
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df
fnl=fnl.replace([np.inf,-np.inf],0)
fnl=fnl.loc[:,~(fnl.columns.duplicated())]
fnl=reduce_mem_usage(fnl)

In [ ]:
fnl.to_csv('prepared.csv')
